# <div align="center"> 机器分配草案 </div>

## 假设

- 同一区域的课表时间统一, 比如该区域都是每天8:10上第一节课, 8:50下课, 课间休息10分钟.

- 不存在课表调休, 比如节假日补课, 老师换课等因特殊不确定原因及不确定调课结果等

- 非上课时间任务强制终止属于正常现象

- 同一个学校, 班级的人数尽量均匀(比如: 都是35人左右)

- 用户训练任务显存占用控制在2GB以内

## 用户编号规范

```

                                           学校    用户序号(学生&教师)
                                           ---    ---
                                        3332220304001
                                        ---   ----
                                        地区   班级(教师)

```

地区码: 000 - 999

学校码: 000 - 999

班级号: 0000 - 9999  (0000:老师, 0304: 3年级4班)

用户号: 000 - 999

## 区域

区域码|名字|上课时间表|课程安排表|用户列表|机器节点
:---:|:---:|:---:|:---:|:---:|:---:
<img width=70px>|<img width=70px/>|<img width=70px/>|<img width=70px/>|<img width=150px/>|<img width=150px/>
101|华东一区| | |[1012010301001, 1012010301002,...,1012010302001] |[192.168.1.101:8989,...,192.168.1.102:8089]
102|华东二区| | |[1022010301001, 1022010301002,...,1022010302001] |[192.168.2.102:8089,...,192.168.2.111:8089]

### 调度表

调度表由指定区域里所有学校的课表汇总形成的, 按1周7天, 每天8节课(24小时), 形成一个时间矩阵,如T(1, 3)表示星期1第3节课, 而T(1, 3)的值为学校3332020302(☺), T(x, y)可以由当前的日期获取到(20200719-08:15:00 --> T(1, 1)), 而T(x, y)的值也是确定的(一个区域的排课安排是已知的).

如:

```
       3332010301: ☻ (编号为333区域201学校3年级1班: 30人)
       3332020302: ☺ (编号为333区域202学校3年级2班: 42人)
       3332030302: ◎ (编号为333区域203学校3年级2班: 29人)

              
                       星    星    星    星    星    星    星 
                       期    期    期    期    期    期    期 
                       一    二    三    四    五    六    日
                  +------------------------------------------------> 周
                  |         
                  |                     
(7:50 - 8:30)  1  |    ☻   
                  |          
               2  |                     ☻,☺            
                  |         
               3  |    ☺     ☺     ☻          ☺ 
                  |                      
               4  |    ☻          ☻,☺         ☺ 
                  |         
               5  |                ☺        ☻,☺,◎ 
                  |         
               6  |         ☺,☻    ☻          ☻
                  |         
               7  |   ☺,☻    ☻           ☻
                  |         
               8  |                
                  |         
                  |
                  v
                时间(节)

```

情景:

1. 每天的课应该不是饱和, 比如`T(1,2), T(5,7)`没有安排课程, 此时`T(1,2)=[], T(5,7)=[]`

2. 同一时刻存在多个班同时上课的情景, 比如`T(4,2)=[2010301, 2020302]` 

3. 按每天24小时算的话, 凡是不在规定课表内的时间, 统一记为`T(0, 0)`, 比如, 周末, 夜晚, 节假日, 即默认`T(x,y)=[]`



### 算力节点数(机器)

1. 按每个任务显存使用不超过2GB, 一台P40大概能运行10个任务, 则1个班40个学生(包括老师), 需要4台P40, 在此人数(40)上, 每增加1-10个人需要另增加1台P40.

2. ~~某区域下调度课表如果同一节课只存在一个班级上课, 则只需一个班级的台数(1台/10), 即为x.~~

3. ~~某区域下调度课表如果同一节课存在Y个班级同时上课, 则这个区域的机器个数按 x * max(Y)~~

4. 某区域的机器使用个数计算公式:

$$
    \bigg ( \max_{x \in X, y \in Y} \sum_{c}^{T(x, y)} S(c) + (K-1) \bigg ) / K
$$

X: 周, 取值为1, 2, 3, 4, 5, 6, 7

Y: 节, 取值为1, 2, 3, 4, 5, 6, 7, 0(表示非上课节)

S(c): 返回c班的学生人数

K: 固定常数, 如果一台P40能够运行10个显存不超过2GB的任务, 则K=10

## 调度表属性

status: 有课busy, 无课free

## 任务属性

status: running(正在运行), stopped(手动停止或者异常停止), paused(暂停), **robbed(资源被抢占)**, finished(正常结束)

## 用户属性

active: 活动, 非活动

## 场景

### 课上拒绝

```
      用户(老师&学生)                 业务服务端                         框架服务                                    
         -------                    ----------                    -----------------         
            |                           |                                 |
            |        a1                 |                                 |                               
            |-------------------------->|                                 |                               
            |        a2                 |                                 |                               
            |<--------------------------|                                 |                               
            |                           |                                 |
                                                                                                          
                                                                                                          
a1: 用户编号userid=3332030301在请求训练

a2: 后台根据当前日期时间获取T(x, y)=[3332010301, 3332020302](调度表busy), 且userid不在T(x,y)中, 拒绝 
```

### 课上清理

```

      用户(老师&学生)                  业务服务端                         框架服务                                    
         -------                    ----------                    -----------------         
            |                           |                                 |                   
            |                           |                a1               |                   
            |                           | <-----------------------------| | 
            |                           |                                 |                   
            |        a2                 |                                 |                   
            |-------------------------->|             a3                  |                   
            |                           |-------------------------------->|                   
            |                           |             a4                  |                   
            |        a5                 |<------------------------------| |                   
            |<--------------------------|                                 |                   
            |                           |                                 |                   
                                                                                              
                                                                                              
a1: 每30s上报一次内存情况                                
                                                                                              
a2: 上课时间, 老师准备上课, 登录网页, 后台根据当前日期时间获取T(x, y)=[3332010301, 3332020302]及需要的资源
                                                                                              
a3: 如果该节课资源不足, 调用框架接口清理资源, 两种方式:
    1. 传给框架需要被清理的任务列表, 框架逐个停止.
    2. 传给框架匹配模式[^3332010301*, ^3332020302*, Teac], 框架根据模式自动清理. (^表示非, *表示任意)                                                                                           
                                                                                              
a4: 框架清理后, 异步方式告知哪些用户的任务被清理掉了.                                                                                              

a5: 如果用户属性处于活动(active)状态, 需要友好提示该用户(任务被抢占)

```

### 资源满足: 课下(后台)正常调度

```
      用户(老师&学生)                  业务服务端                          框架服务
         -------                    ----------                    -----------------
            |                           |                a1               |
            |                           | <-----------------------------| |
            |                           |                                 |
            |                           |                                 |
            |                           |----+                            |
            |                           |    | a2: check T(x, y)          |
            |                           |    |                            |
            |                           |<---+                            |
            |                           |               a3: start         |
            |                           |-------------------------------->|
            |                           |                                 |
            |                           |                                 |
            |                           |                                 |
            |                           |                                 |


a1: 框架周期上报算力资源信息

a2: 定时(或者触发方式)根据当前日期时间获取T(x, y), 如果`T(x, y)=[]`, 说明调度表此刻为free状态

a3: 资源充足, 获取该区域内的所有用户任务及用户active属性, 并根据算力信息, 进行后台任务调度(active优先)
```

### 资源不足: 课下(后台)抢占调度

```
      用户(老师&学生)                 业务服务端                          框架服务
         -------                    ----------                    -----------------
            |                           |                a1               |
     active |                           | <-----------------------------| |
            |                           |                                 |
            |                           |                                 |
            |                           |----+                            |
            |                           |    | a2: check T(x,y)           |
            |                           |    |                            |
            |                           |<---+                            |
            |                           |                                 |
            |                      +----|                                 |
            |          a3: active  |    |                                 |
            |                      |    |                                 |
            |                      +--->|               a4: pause         |
            |                           | ------------------------------> |
            |                           |      xxx: paused                |
            |                           | <-----------------------------| |
            |                           |                                 |
            |                           |               a5: start         |
            |                           | ------------------------------> |
            |                           |                                 |
            |                           | <-----------------------------| |
            |                           |                                 |


a1: 框架周期上报算力资源信息

a2: 定时(或者触发方式)根据当前日期时间获取T(x, y), 如果`T(x, y)=[]`, 说明调度表此刻为free状态

a3: 资源不足, 且用户状态为active时, 查询该区域所有unactive状态的后台任务

a4: 根据资源需要情况进行puase(抢占)掉unactive用户状态的后台任务, 为active用户任务准备资源

a5: 资源满足, 启动active用户的任务

```

## 调度表 

### 课表时间安排

In [1]:
from IPython.display import Markdown
import time

timetable = (
    (1, "08:10-08:50"),
    (2, "09:00-09:40"),
    (3, "10:00-10:40"),
    (4, "10:50-11:30"),
    (5, "14:10-14:50"),
    (6, "15:00-15:30")
)

mdtable = '''
| 节 | 时间 |
|:---: | :---: |
|<img width="200px"/> | <img width="600px"/> |
'''

for y, timestr in timetable:
    mdtable += f'|{y}|{timestr}\n'
    
Markdown(mdtable)


| 节 | 时间 |
|:---: | :---: |
|<img width="200px"/> | <img width="600px"/> |
|1|08:10-08:50
|2|09:00-09:40
|3|10:00-10:40
|4|10:50-11:30
|5|14:10-14:50
|6|15:00-15:30


### 课表班级安排

In [2]:
schedtable = (
    (1, 2, ['7010301']),
    (1, 5, ['7010302', '7020301']),
    (2, 3, ['7010303']),
    (2, 4, ['7020302']),
    (2, 6, ['7010304', '7030301']),
    (3, 5, ['7020303', '7030302']),
    (4, 1, ['7030303']),
    (5, 2, ['7020304']),
    (5, 5, ['7030304']),
) 

mdtable = '''
时间 | 节 | 星期一 | 星期二 | 星期三 | 星期四 | 星期五 | 星期六 | 星期日
:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:
'''

for imgwd in range(9):
    mdtable += '<img width=80px/>|'
mdtable +='\n'
for y, timestr in timetable:
    mdtable += f'{timestr}|{y}'
    for x in (1, 2, 3, 4, 5, 6, 0):
        cs = []
        for x_, y_, cs_ in schedtable:
            if x_ == x and y_ == y:
                cs = cs_
                break
        if len(cs) == 0:
            mdtable += '|[      ]'
        else:
            mdtable += '|{}'.format(cs)
    mdtable += '\n'    
Markdown(mdtable)


时间 | 节 | 星期一 | 星期二 | 星期三 | 星期四 | 星期五 | 星期六 | 星期日
:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:
<img width=80px/>|<img width=80px/>|<img width=80px/>|<img width=80px/>|<img width=80px/>|<img width=80px/>|<img width=80px/>|<img width=80px/>|<img width=80px/>|
08:10-08:50|1|[      ]|[      ]|[      ]|['7030303']|[      ]|[      ]|[      ]
09:00-09:40|2|['7010301']|[      ]|[      ]|[      ]|['7020304']|[      ]|[      ]
10:00-10:40|3|[      ]|['7010303']|[      ]|[      ]|[      ]|[      ]|[      ]
10:50-11:30|4|[      ]|['7020302']|[      ]|[      ]|[      ]|[      ]|[      ]
14:10-14:50|5|['7010302', '7020301']|[      ]|['7020303', '7030302']|[      ]|['7030304']|[      ]|[      ]
15:00-15:30|6|[      ]|['7010304', '7030301']|[      ]|[      ]|[      ]|[      ]|[      ]


### API: 初始化T(x, y)

In [3]:
start_end_timing = []
for y, timestr in timetable:
    ss, es = timestr.replace(':', '').split('-')
    start_end_timing.append((y, int(ss), int(es)))
start_end_timing

[(1, 810, 850),
 (2, 900, 940),
 (3, 1000, 1040),
 (4, 1050, 1130),
 (5, 1410, 1450),
 (6, 1500, 1530)]

In [4]:
def InitTxy(schedtable):
    Y = 0
    for _, y, _ in schedtable:
        if y > Y:
            Y = y
    # 初始化空表
    T = [[[] for _ in range(Y+1)] for _ in range(7)]
    # 课程
    for x, y, cs in schedtable:
        T[x][y] = cs
        
    return T
T = InitTxy(schedtable)
T

[[[], [], [], [], [], [], []],
 [[], [], ['7010301'], [], [], ['7010302', '7020301'], []],
 [[], [], [], ['7010303'], ['7020302'], [], ['7010304', '7030301']],
 [[], [], [], [], [], ['7020303', '7030302'], []],
 [[], ['7030303'], [], [], [], [], []],
 [[], [], ['7020304'], [], [], ['7030304'], []],
 [[], [], [], [], [], [], []]]

### API: 计算给定日期的T(x, y)

In [5]:
def Date2Txy(T, timetable, timesecs=None):
    if timesecs is None:
        timesecs = time.time()
    xystr = time.strftime('%w %H%M', time.localtime(timesecs))
    xstr, ystr = xystr.split()
    x, y, yint = int(xstr), 0, int(ystr)
    for c, st, et in timetable[::-1]: # reverse order
        if yint >= st:
            y = c if yint < et else 0
            break
    if y == 0:
        x = 0
    print(xystr, f'T({x},{y})')
    return T[x][y]
Date2Txy(T, start_end_timing)
timetable[::-1]

2 1617 T(0,0)


((6, '15:00-15:30'),
 (5, '14:10-14:50'),
 (4, '10:50-11:30'),
 (3, '10:00-10:40'),
 (2, '09:00-09:40'),
 (1, '08:10-08:50'))

#### 测试

In [6]:
test_times = (
    "2020-07-20 08:06:05",  # T(0, 0)
    "2020-07-23 08:10:05",  # T(4, 1)
    "2020-07-20 08:51:05",  # T(0, 0)
    "2020-07-20 09:06:05",  # T(1, 2)
    "2020-07-15 09:53:05",  # T(0, 0)
    "2020-07-21 10:16:05",  # T(2, 3)
    "2020-07-21 11:22:05",  # T(2, 4)
    "2020-07-18 12:21:05",  # T(0, 0)
    "2020-07-20 14:23:05",  # T(1, 5)
    "2020-07-22 14:58:05",  # T(0, 0)
    "2020-07-21 15:27:05",  # T(2, 6)
    "2020-07-13 17:27:05",  # T(0, 0)
)

for timestr in test_times:
    print(timestr, Date2Txy(T, start_end_timing, time.mktime(time.strptime(timestr,'%Y-%m-%d %H:%M:%S'))))

1 0806 T(0,0)
2020-07-20 08:06:05 []
4 0810 T(4,1)
2020-07-23 08:10:05 ['7030303']
1 0851 T(0,0)
2020-07-20 08:51:05 []
1 0906 T(1,2)
2020-07-20 09:06:05 ['7010301']
3 0953 T(0,0)
2020-07-15 09:53:05 []
2 1016 T(2,3)
2020-07-21 10:16:05 ['7010303']
2 1122 T(2,4)
2020-07-21 11:22:05 ['7020302']
6 1221 T(0,0)
2020-07-18 12:21:05 []
1 1423 T(1,5)
2020-07-20 14:23:05 ['7010302', '7020301']
3 1458 T(0,0)
2020-07-22 14:58:05 []
2 1527 T(2,6)
2020-07-21 15:27:05 ['7010304', '7030301']
1 1727 T(0,0)
2020-07-13 17:27:05 []


## 调度系统设计

### 区域信息模板

In [7]:
# 区域信息
REGIONS = {
    # 地区101
    '101': {
        'time_table': { # 时间表
            # 第一节课: [开始时间, 结束时间]
            1: ['08:10', '08:50'],
            2: ['09:00', '09:40'],
            3: ['10:00', '10:40'],
            4: ['10:50', '11:30'],
            5: ['14:10', '14:50'],
            6: ['15:00', '16:00']
        },
        'class_table': { # 课程表
            # 星期一
            1: {
                # 第２节课
                2: ['7010301'],
                # 第5节课
                5: ['7010302', '7020301'],
            },
            # 星期二
            2: {
                3: ['7010303'],
                4: ['7020302'],
                6: ['7010304', '7030301'],
            },
            # 星期三
            3: {
                5: ['7020303', '7030302'],
            },
            # 星期四
            4: {
                1: ['7030303'],
            },
            # 星期五
            5: {
                2: ['7020304'],
                5: ['7030304'],
            },
        },
        # 用户列表
        'user_list': [
            '1017010301001',
            '1017010301002',
            '1017010302001',
            '1017010302002',
            '1017010303001',
            '1017010303002',
            '1017010304001',
            '1017010304002',
            
            '1017020301001',
            '1017020301002',
            '1017020302001',
            '1017020302002',
            '1017020303001',
            '1017020303002',
            '1017020304001',
            '1017020304002',
            
            '1017030301001',
            '1017030301002',
            '1017030302001',
            '1017030302002',
            '1017030303001',
            '1017030303002',
            '1017030304001',
            '1017030304002',
        ],
        # GPU服务器节点
        'server_nodes': [
            '10.255.0.58:8119',
            '10.255.0.202:8119',
        ]
    },
    # 地区102
    '102': {
        'time_table': { # 时间表
            1: ['08:10', '08:50'],
            2: ['09:00', '09:40'],
            3: ['10:00', '10:40'],
            4: ['10:50', '11:30'],
            5: ['14:10', '14:50'],
            6: ['15:00', '15:30']
        },
        'class_table': { # 课程表
            1: {
                2: ['5010301'],
                4: ['5020302'],
                5: ['5010302', '5020301'],
            },
            2: {
                6: ['5010304', '5030301'],
            },
            3: {
                3: ['5010303'],
                5: ['5020303', '5030302'],
            },
            4: {
                1: ['5030303'],
                2: ['5020304'],
            },
            5: {
                5: ['5030304'],
            },
        },
        # 用户列表
        'user_list': [
            '1025010301001',
            '1025010301002',
            '1025010302001',
            '1025010302002',
            '1025010303001',
            '1025010303002',
            '1025010304001',
            '1025010304002',
            
            '1025020301001',
            '1025020301002',
            '1025020302001',
            '1025020302002',
            '1025020303001',
            '1025020303002',
            '1025020304001',
            '1025020304002',
            
            '1025030301001',
            '1025030301002',
            '1025030302001',
            '1025030302002',
            '1025030303001',
            '1025030303002',
            '1025030304001',
            '1025030304002',
        ],
        # GPU服务器节点
        'server_nodes': [
            '10.255.0.201:8119',
        ]
    }
}
# REGIONS

#### 区域101课程时间表

In [8]:
timetable = REGIONS['101']['time_table']

mdtable = '''
| 节 | 时间 |
|:---: | :---: |
|<img width="200px"/> | <img width="600px"/> |
'''

for clsid, timestr in timetable.items():
    mdtable += f'|{clsid}|{timestr}\n'
Markdown(mdtable)


| 节 | 时间 |
|:---: | :---: |
|<img width="200px"/> | <img width="600px"/> |
|1|['08:10', '08:50']
|2|['09:00', '09:40']
|3|['10:00', '10:40']
|4|['10:50', '11:30']
|5|['14:10', '14:50']
|6|['15:00', '16:00']


#### 区域101课程安排表

In [9]:
schedtable = REGIONS['101']['class_table']

mdtable = '''
节 | 星期一 | 星期二 | 星期三 | 星期四 | 星期五 | 星期六 | 星期日
:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:
'''

for imgwd in range(8):
    mdtable += '<img width=80px/>|'
mdtable +='\n'

mat = [[[] for _ in range(7)] for _ in range(6)]
for w, cinfo in schedtable.items():
    for clsid, clses in cinfo.items():
        mat[clsid-1][w-1] = clses
        
for idx, row in enumerate(mat, 1):
    mdtable += f'{idx}'
    for it in row:
        mdtable += '|{}'.format(it)
    mdtable += '\n'
            
Markdown(mdtable)


节 | 星期一 | 星期二 | 星期三 | 星期四 | 星期五 | 星期六 | 星期日
:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:
<img width=80px/>|<img width=80px/>|<img width=80px/>|<img width=80px/>|<img width=80px/>|<img width=80px/>|<img width=80px/>|<img width=80px/>|
1|[]|[]|[]|['7030303']|[]|[]|[]
2|['7010301']|[]|[]|[]|['7020304']|[]|[]
3|[]|['7010303']|[]|[]|[]|[]|[]
4|[]|['7020302']|[]|[]|[]|[]|[]
5|['7010302', '7020301']|[]|['7020303', '7030302']|[]|['7030304']|[]|[]
6|[]|['7010304', '7030301']|[]|[]|[]|[]|[]


### 消息分发及处理接口

In [10]:
from k12libs.utils.nb_easy import k12ai_train_execute
from typing import Optional, Union
from schedule import Scheduler
from abc import ABC, abstractmethod
from threading import Thread
import requests
import redis
import time
import json

DEBUG = True

if DEBUG:
    ROBBED_USERS = []
    
K12AI_DEFAULT_SERVER='10.255.0.58:8119'
REDIS_HOST='10.255.0.58'
REDIS_PORT=10090

class IMessageDispatcher(ABC):
    @abstractmethod
    def dispatch_message(message:dict):
        """
        (M) 分发消息
        """

class IMessageHandler(ABC):
    @abstractmethod
    def handle_message(message:dict):
        """
        (M) 处理消息
        """

### 远端框架RestAPI

In [11]:
class RemoteAPI(object):
    @staticmethod
    def restapi_post(server, uri, data):
        api = 'http://%s/%s' % (server, uri)
        try:
            response = requests.post(url=api, json=data, timeout=10)
            result = json.loads(response.text)
            print(result)
            if 'error' in result:
                return None
            return result
        except Exception as err:
            print('{}'.format(err))
            return None
    
    @staticmethod
    def k12ai_get_task(server, user='*', uuid='*'):
        """
        获取所给节点下运行的用户任务
        
        Args:
            server: GPU服务器节点
            user: 指定用户ID, 不指定获取所有
            uuid: 指定任务ID, 不指定获取所有
            
        Return:
            response: 框架返回执行结果
        """
        
        data = {
            'op': 'query',
            'user': user,
            'service_uuid': uuid,
            'service_params': {'services': True},
            'async': False
        }
        task_list = []
        response  = RemoteAPI.restapi_post(server, 'k12ai/platform/stats', data)
        print(response)
        if response: 
            for info in response['data']['services']:
                task_list.append(info)
        return task_list

    @staticmethod
    def k12ai_rob_task(server, user, uuid):
        """
        停止(抢占资源)任务
        
        Args:
            server: GPU服务器
            user: 指定用户ID, 不指定获取所有
            uuid: 指定任务ID, 不指定获取所有
            
        Return:
            response: 框架返回执行结果
        """
        
        data = {
            'token': '123456',
            'op': 'train.rob',
            'user': user,
            'service_name': 'k12cv',
            'service_uuid': uuid,  
        }
        
        if DEBUG: # 调试 记录被强制停止的用户任务
            ROBBED_USERS.append({
                'user': user,
                'uuid': uuid,
                'server': server
            })
        return RemoteAPI.restapi_post(server, 'k12ai/framework/execute', data)
    
    @staticmethod
    def k12ai_run_task(server, user, uuid):
        """
        启动任务(状态为robbed)
        
        Args:
            server: GPU服务器
            user: 指定用户ID, 不指定获取所有 
            uuid: 指定任务ID, 不指定获取所有
            
        Return:
            response: 框架返回执行结果
        """
        
        data = {
            'token': '123456',
            'op': 'train.resume',
            'user': user,
            'service_name': 'k12cv',
            'service_uuid': uuid,  
        }
        if DEBUG: # 调试 启动任务后, 从记录表中移除用户任务
            ROBBED_USERS.remove({
                'user': user,
                'uuid': uuid,
                'server': server
            })
        return RemoteAPI.restapi_post(server, 'k12ai/framework/execute', data)
    

In [12]:
# 模拟启动用户任务
# k12ai_train_execute(user='1017020302001', epoch_num=100)
# k12ai_train_execute(user='1017010302001', epoch_num=100)
# k12ai_train_execute(user='1017020303001', epoch_num=100)

In [13]:
# 测试API
task_list = RemoteAPI.k12ai_get_task(K12AI_DEFAULT_SERVER)
for info in task_list:
    print(RemoteAPI.k12ai_rob_task(K12AI_DEFAULT_SERVER, info['user'], info['service_uuid']))
    time.sleep(5)
    print(RemoteAPI.k12ai_run_task(K12AI_DEFAULT_SERVER, info['user'], info['service_uuid']))

{'code': 100000, 'message': {'en': 'success', 'cn': '成功'}, 'data': {'services': [{'id': '164aae2ef438', 'op': 'train.resume', 'user': '1017020303001', 'service_uuid': '211d2c-0', 'service_pid': 12497, 'service_starttime': '2020-08-04T08:16:43.509418874Z'}]}}
{'code': 100000, 'message': {'en': 'success', 'cn': '成功'}, 'data': {'services': [{'id': '164aae2ef438', 'op': 'train.resume', 'user': '1017020303001', 'service_uuid': '211d2c-0', 'service_pid': 12497, 'service_starttime': '2020-08-04T08:16:43.509418874Z'}]}}
{'code': 100000, 'message': {'en': 'success', 'cn': '成功'}}
{'code': 100000, 'message': {'en': 'success', 'cn': '成功'}}
{'code': 100000, 'message': {'en': 'success', 'cn': '成功'}}
{'code': 100000, 'message': {'en': 'success', 'cn': '成功'}}


### 实现Redis读取及消息分发

In [14]:
class RedisDispatcher(IMessageDispatcher, Thread):
    """
    接收框架传来的消息, 并分发到对应的接收者去处理.
    """
    def __init__(self):
        Thread.__init__(self)
        super(RedisDispatcher, self).__init__()
        class RedisQueue(object):
            def __init__(self, host, port, pswd, keys):
                self.keys = keys
                self.db= redis.StrictRedis(
                    host=host,
                    port=port,
                    password=pswd)

            def __next__(self):
                return self.db.blpop(self.keys)
            
        self.queue = RedisQueue(
            host=REDIS_HOST,
            port=REDIS_PORT,
            pswd='qY3Zh4xLPZNMkaz3',
            keys=['k12ai.resource', 'k12ai.status'])
        self.handlers = {}
    
    def register_handler(self, msg_type: str, msg_handler: IMessageHandler):
        """
        注册消息处理者
        
        Args:
            msg_type: [M] 接收者关注的消息类型
            msg_handler: [M] 消息处理者, 类型必须为IMessageHandler
        """
        
        if msg_type not in self.handlers.keys():
            self.handlers[msg_type] = []
        self.handlers[msg_type].append(msg_handler)
        
    
    def dispatch_message(self, msg: dict) -> None:
        """
        根据不同的消息类型进行分发消息
        
        Args:
            message: 来自框架节点的消息
        """
        
        # 检查消息的合法型
        if 'type' not in msg.keys():
            print(f'message content is not invalid!')
            return
        
        # 根据类型将消息发送到对应的接受者
        if msg['type'] in self.handlers.keys():
            for x in self.handlers[msg['type']]:
                x.handle_message(msg)
        
    def run(self):
        if DEBUG:
            i = 5
        while True:
            item = next(self.queue)
            if item and len(item) == 2:
                self.dispatch_message(json.loads(item[1]))
            if DEBUG:
                if i < 0:
                    break
                i -= 1
        print(f'{self.__class__.__name__} instance quit!')

### 实现机器调度及消息处理

In [15]:
class ServerNodeScheduler(IMessageHandler, Scheduler, Thread):
    # 区域信息
    REGION_KEYS = [
        'time_table',   # 时间表, 每节课的上课时间和结束时间
        'class_table',  # 课程表, 每天AI课第几节课上及哪些班级上课
        'server_nodes', # 服务器节点, 该区域的学生只能分布在这些机器上运行任务
        'user_list'     # 用户列表, 该区域下所有的用户编号
    ]
    # 规定一天的开始时间, 规定一天的结束时间
    MORNING_TIME, EVENING_TIME = '08:00', '18:00',
    # 周属性名字
    WEEKDAYS = ('sunday', 'monday', 'tuesday',
        'wednesday', 'thursday', 'friday', 'saturday')
    
    CPU_THRESHOLD_MB = 5000
    GPU_THRESHOLD_MB = 5000

    def __init__(self, regions:dict, remote: RemoteAPI):
        Thread.__init__(self)
        super(ServerNodeScheduler, self).__init__()
        
        self._remote = remote
        
        # IP映射区域码
        self._server_code_map = {}
        for code, infos in regions.items():
            # 检查是否包含全部信息
            if any([True for x in infos.keys() if x not in self.REGION_KEYS]):
                raise RuntimeError(f'区域-{code}信息缺少必要信息!')               
            for node in infos['server_nodes']:
                self._server_code_map[node] = code

        # 区域调度表
        self._stab = self._init_stab(regions)
        # 区域定时任务
        self._jobs = self._init_jobs(regions)
        
    def _init_stab(self, regions) -> list:
        """
        初始化调度表STab
        
        Args:
            regions: (M) 区域(dict)信息
            
        Return:
            STab: 课程调度矩阵和课程时间编码 {'mat': [[[]]], 'cls':[()], 'u2n': {}, 'sns':[]}
        """
        
        stab = {} 
        for code, infos in regions.items():
            max_clsid = max(infos['time_table'].keys())
            # 初始化空调度矩阵
            schedtable = [[[] for _ in range(max_clsid+1)] for _ in range(len(self.WEEKDAYS))]
            # 填充班级信息
            for wday, courses in infos['class_table'].items():
                for clsid, clses in courses.items():
                    schedtable[wday][clsid] = clses
                    
            # 字符串时间编码为整型: {1: ['08:10', '08:40']} -> [(1, 810, 840)]
            timeencode = []
            for clsid, (ststr, etstr) in infos['time_table'].items():
                stis, etis = ststr.replace(':', ''), etstr.replace(':', '')
                timeencode.append((clsid, int(stis), int(etis)))
            # 逆序(优化性能)
            timeencode.sort(reverse=True)
            
            # 用户号映射机器节点
            u2n, num = {}, len(infos['server_nodes'])
            for idx, userid in enumerate(infos['user_list']):
                u2n[userid] = idx % num
            
            stab[code] = {
                'mat': schedtable,            # 对上课下课定时触发
                'cls': timeencode,            # 结合mat, 判断当前时间是否有课在上, 拒绝非法用户 
                'u2n': u2n,                   # 用户编号转换为对应的机器节点
                'sns': infos['server_nodes'], # 该区域下所有机器节点
            }
            
        return stab
    
    def _everyday(self, wday: Optional[int] = None):
        """
        周一到周日, 创建定时对象, 根据传递的did创建具体周几的对象
        
        Args:
            wday: (O), 0-6分别表示周日, 周一到周六
            
        Return:
            Job: job对象
        """
        
        if wday:
            return getattr(self.every(), self.WEEKDAYS[wday])
        else:
            return self.every().day
    
    def _init_jobs(self, regions) -> list:
        """
        初始化定时(cron)调度任务
        
        Args:
            regions: (M) 区域(dict)信息
        """
        
        if DEBUG: # 调试
            today = time.localtime(time.time())
            MORNING_TIME = time.strftime('%H:%M:%S', time.localtime(time.time() +  2))
            EVENING_TIME = time.strftime('%H:%M:%S', time.localtime(time.time() + 12)) 
            stime = time.strftime('%H:%M:%S', time.localtime(time.time() + 60 - today.tm_sec))
            etime = time.strftime('%H:%M:%S', time.localtime(time.time() + 96 - today.tm_sec))
            print(f'早上时间:{MORNING_TIME} 晚上时间:{EVENING_TIME} 上课时间:{stime} 下课时间:{etime}')
            self._everyday().at(MORNING_TIME).do(
                lambda *args: self.on_good_morning(*args))
            self._everyday().at(EVENING_TIME).do(
                lambda *args: self.on_good_evening(*args))
            self._everyday(today.tm_wday+1).at(stime).do(
                lambda *args: self.on_attend_class(*args),
                101, ['7010304'], ['10.255.0.58:8119'])
            self._everyday(today.tm_wday+1).at(etime).do(
                lambda *args: self.on_finish_class(*args),
                101, ['7010304'], ['10.255.0.58:8119'])
            return
        
        self._everyday().at(MORNING_TIME).do(
            lambda *args: self.on_good_morning(*args))
        self._everyday().at(EVENING_TIME).do(
            lambda *args: self.on_good_evening(*args))
        
        for code, infos in regions.items():
            nodes = infos['server_nodes']
            for wday, courses in infos['class_table'].items():
                for clsid, clses in courses.items(): # 第几节课, 哪些班
                    stime, etime = infos['time_table'][clsid] # 上课时间, 下课时间
                    # 注册上课时准备任务
                    self._everyday(wday).at(stime).do(
                        lambda *args: self.on_attend_class(*args),
                        code, clses, nodes)
                    # 注册下课后处理任务
                    self._everykday(wday).at(etime).do(
                        lambda *args: self.on_finish_class(*args),
                        code, clses, nodes)
                    
        # parent property
        return self.jobs
    
    def _stab_mat_classes(self, region_code, timesecs=None) -> list:
        """
        获取当前(或指定)时间下调度矩阵的上课班级
        
        Args:
            region_code: 区域码
            timesecs: (O) 时间秒, 如果不传使用当前时间
            
        Return:
            classes: 班级列表或者空列表[]
        """
        
        # 检测区域码
        if region_code not in self._stab.keys():
            raise RuntimeError('区域码错误!')
        stab = self._stab[region_code]
        
        # 时间编码
        if timesecs is None:
            timesecs = time.time()
        ltime = time.localtime(timesecs)
        ytime = ltime.tm_hour*100 + ltime.tm_min
        
        # 判断当前属于哪节课
        for c, st, et in stab['cls']:
            if ytime >= st:
                if ytime < et:
                    return stab['mat'][ltime.tm_wday+1][c]
                return []
        return []
    
    def _stab_status_free(self, region_code, timesecs=None) -> bool:
        """
        检测当前(或指定)时间下调度表的状态(busy or free)是否空闲
        
        Args:
            region_code: 区域码
            timesecs: (O) 时间秒, 如果不传使用当前时间
            
        Return:
            bool
        """
        return 0 == len(self._stab_mat_classes(region_code, timesecs))
                    
    def on_good_morning(self):
        """
        一天的开始, 处理这一天的预备事务
        """
        print('早上好')
    
    def on_good_evening(self):
        """
        一天的结束, 处理这一天的清理事务
        """
        print('晚上好')
        
    def on_attend_class(self, region_code, class_list, server_nodes):
        """
        上课事件, 将非上课班级的学生资源释放掉
        
        Args:
            region_code: 区域码
            class_list: 班级列表
            server_nodes: 该区域的训练机器
        """
        print("上课事件", region_code, class_list, server_nodes)
        
        for server in server_nodes:
            for info in self._remote.k12ai_get_task(server):
                user = info['user']
                uuid = info['service_uuid']
                code, cls = user[:3], user[3:10]
                # 非上课班级的用户, 强制抢占
                if code != region_code or cls not in class_list:
                    self._remote.k12ai_rob_task(server, user, uuid)
                    print(f'任务({server}/{user}/{uuid})被抢占')
    
    def on_finish_class(self, region_code, class_list, server_nodes):
        """
        下课事件, 可以启动一些被抢占的任务(robbed)执行, 暂时可以不处理
        
        Args:
            region_code: 区域码
            class_list: 班级列表
            server_nodes: 该区域的训练机器
        """
        print('下课事件', region_code, class_list, server_nodes)
        
    def handle_message(self, message):
        """
        接收来自框架发来的服务节点的资源信息(CPU, GPU的使用情况), 如果资源充足, 启动robbed任务
        """
        
        print(message)
        
        server = message['server']
        data = message['data']['data']
        
        # 检查消息的合法性
        if not all(x in data.keys() for x in ('cpu_memory_free_MB', 'gpus')):
            print('消息缺少关键内容')
            return
        if 'gpu_memory_free_MB' not in data['gpus']:
            print('消息缺少关键内容')
            return
        
        # 判断资源是否充足
        cpu_free_mb = data['cpu_memory_free_MB']
        gpu_free_mb = data['gpus']['gpu_memory_free_MB']
        if gpu_free_mb < self.GPU_THRESHOLD_MB and cpu_free_mb < self.CPU_THRESHOLD_MB:
            print('资源不足, 不做处理')
            return
        
        # 判断是否可以启动robbed任务(当前没有班级正在上课)
        region_code = self._server_code_map.get(server, None)
        if not (region_code and self._stab_status_free(region_code)):
            print('服务器不在任何区域内或者当前调度课程繁忙(在上课)')
            return
        
        # 查询一个robbed状态的用户(active用户优先)
        # TODO 提供接口
        for item in ROBBED_USERS:
            if item['server'] == server:
                # 恢复训练
                user, uuid = item['user'], item['uuid']
                print(f'resume the robbed task:{user}-{uuid} on {server}')
                self._remote.k12ai_run_task(server, user, uuid)
                break

    def run(self):
        while True:
            self.run_pending()
            if DEBUG:
                datetime = time.strftime('%H:%M:%S', time.localtime(time.time()))
                print(f'now {datetime}, sleep: {self.idle_seconds} seconds')
                if self.idle_seconds > 900:
                    break
            time.sleep(self.idle_seconds) # the next job interval time seconds
        print(f'{self.__class__.__name__} instance quit!')

## 测试

In [16]:
# 模拟启动用户任务
k12ai_train_execute(user='1017020302001', epoch_num=100)
k12ai_train_execute(user='1017010302001', epoch_num=100)
k12ai_train_execute(user='1017020303001', epoch_num=100)

['framework/1017020303001/211d2c-0/train']

In [17]:
xscheduler = ServerNodeScheduler(REGIONS, RemoteAPI())
xscheduler.start()
dispatcher = RedisDispatcher()
dispatcher.register_handler('resource', xscheduler)
dispatcher.start()

早上时间:16:17:44 晚上时间:16:17:54 上课时间:16:18:00 下课时间:16:18:36
now 16:17:42, sleep: 1.621923 seconds
早上好
now 16:17:44, sleep: 9.999103 seconds
晚上好
now 16:17:54, sleep: 5.99174 seconds
上课事件 101 ['7010304'] ['10.255.0.58:8119']
{'code': 100000, 'message': {'en': 'success', 'cn': '成功'}, 'data': {'services': [{'id': '49ce9cc958bd', 'op': 'train.start', 'user': '1017010302001', 'service_uuid': '211d2c-0', 'service_pid': 13992, 'service_starttime': '2020-08-04T08:17:37.913531506Z'}, {'id': '9155627a1393', 'op': 'train.start', 'user': '1017020302001', 'service_uuid': '211d2c-0', 'service_pid': 13652, 'service_starttime': '2020-08-04T08:17:29.802836094Z'}]}}
{'code': 100000, 'message': {'en': 'success', 'cn': '成功'}, 'data': {'services': [{'id': '49ce9cc958bd', 'op': 'train.start', 'user': '1017010302001', 'service_uuid': '211d2c-0', 'service_pid': 13992, 'service_starttime': '2020-08-04T08:17:37.913531506Z'}, {'id': '9155627a1393', 'op': 'train.start', 'user': '1017020302001', 'service_uuid': '211d2c